In [ ]:
# pip install cryptography   # (if you don't have it already)

In [1]:
from cryptography.hazmat.primitives.asymmetric import x25519
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
import os

print("ECDH (X25519) Key Exchange Demo\n" + "="*50)

# 1. Alice generates her private/public key
alice_private_key = x25519.X25519PrivateKey.generate()
alice_public_key  = alice_private_key.public_key()

# 2. Bob generates his private/public key
bob_private_key = x25519.X25519PrivateKey.generate()
bob_public_key  = bob_private_key.public_key()

# 3. They exchange ONLY the public keys (32 bytes each)
print(f"Alice public key : {alice_public_key.public_bytes_raw().hex()}")
print(f"Bob   public key : {bob_public_key.public_bytes_raw().hex()}")

# 4. Both compute the exact same shared secret
shared_secret_alice = alice_private_key.exchange(bob_public_key)
shared_secret_bob   = bob_private_key.exchange(alice_public_key)

assert shared_secret_alice == shared_secret_bob
print(f"\nShared secret (32 bytes): {shared_secret_alice.hex()}")

# 5. Turn the raw shared secret into a proper AES-256 key using HKDF
hkdf = HKDF(
    algorithm=hashes.SHA256(),   # ← this is the correct import
    length=32,
    salt=None,                   # or os.urandom(16) for extra randomness
    info=b'handshake data',      # optional context
)
aes256_key = hkdf.derive(shared_secret_alice)

print(f"\nDerived AES-256 key      : {aes256_key.hex()}")
print("Both parties now have the same 256-bit symmetric key – ready for AES-GCM, ChaCha20, etc.!")

ECDH (X25519) Key Exchange Demo
Alice public key : 8c674675f2b80d354db0905ca6eb379c263db0ae902de6cc177b6c6c2c48667c
Bob   public key : 01ec40f61185f8464e558a75bf695ea8ba1c58e64bd681bd93e5b3081e9ecf74

Shared secret (32 bytes): f1e6a167cf0068a8d00e3c7fe1ddfbc6aa0d276000ad5c1b2b12b1732e333c37

Derived AES-256 key      : 36201e97f7376c04e28a5323a285ae078049e45a39d157b59d1ae0d6ddeaa87a
Both parties now have the same 256-bit symmetric key – ready for AES-GCM, ChaCha20, etc.!
